In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import path
import matplotlib.pyplot as plt
import time
import xarray as xr
from shapely.geometry import Point

In [2]:
# Load ERA5 data
filename = r".\raw\data_stream-wave_stepType-instant.nc"
era5 = xr.open_dataset(filename)
era5=era5.sel(valid_time=slice('2020-01-01','2023-12-31')) #'2021-01-01','2021-12-31'
era5.load()

<xarray.Dataset> Size: 5MB
Dimensions:     (valid_time: 35064, latitude: 3, longitude: 5)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 281kB 2020-01-01 ... 2023-12-31T2...
  * latitude    (latitude) float64 24B 29.0 28.5 28.0
  * longitude   (longitude) float64 40B -96.0 -95.5 -95.0 -94.5 -94.0
    expver      (valid_time) <U4 561kB '0001' '0001' '0001' ... '0001' '0001'
Data variables:
    mwp         (valid_time, latitude, longitude) float32 2MB nan nan ... 3.925
    swh         (valid_time, latitude, longitude) float32 2MB nan nan ... 0.7233
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-15T14:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
bbox_filepath = r".\bbox_wave.geojson"
bbox_gdf = gpd.read_file(bbox_filepath)

era5 = era5.copy()
if "time" not in era5.coords:
    era5 = era5.rename({"valid_time": "time"})

era5 = era5.rename({
    "swh": "hs",
    "mwp": "tp",
    "longitude": "lon",
    "latitude": "lat"
})
era5 = era5.set_coords(["lon", "lat"])

In [4]:
# Filter points within the bounding box
lon_flat, lat_flat = np.meshgrid(era5["lon"], era5["lat"])
points_gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(lon_flat.ravel(), lat_flat.ravel()),
    crs="EPSG:4326"
)

points_within_bbox = points_gdf[points_gdf.within(bbox_gdf.unary_union)]
era5 = era5.sel(
    lon=xr.DataArray(points_within_bbox.geometry.x.values, dims="index"),
    lat=xr.DataArray(points_within_bbox.geometry.y.values, dims="index"),
    method="nearest"
)

C:\Users\ferna\AppData\Local\Temp\ipykernel_59304\1625525427.py:8: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  points_within_bbox = points_gdf[points_gdf.within(bbox_gdf.unary_union)]


In [5]:
# drop values with NaNs - you should end up only with the offshore points that are part of your bounding box
era5 = era5.dropna(dim='index')

In [6]:
era5.load()

<xarray.Dataset> Size: 1MB
Dimensions:  (time: 35064, index: 1)
Coordinates:
    number   int64 8B 0
  * time     (time) datetime64[ns] 281kB 2020-01-01 ... 2023-12-31T23:00:00
    lat      (index) float64 8B 29.0
    lon      (index) float64 8B -95.0
    expver   (time) <U4 561kB '0001' '0001' '0001' ... '0001' '0001' '0001'
Dimensions without coordinates: index
Data variables:
    tp       (time, index) float32 140kB 3.509 3.478 3.457 ... 3.737 3.823 3.899
    hs       (time, index) float32 140kB 0.3714 0.3467 0.3243 ... 0.4732 0.4858
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-15T14:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
# Observation coordinates
obs_coord = [28.935,-95.295]  # (lat, lon)

In [9]:
# Function to find the nearest index for a given value in an array
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

# Combine 'lat' and 'lon' into a single DataFrame to find the nearest point
lat_values = era5['lat'].values  # Latitude values
lon_values = era5['lon'].values  # Longitude values

# Calculate the Euclidean distance for each (lat, lon) pair
distances = np.sqrt((lat_values - obs_coord[0])**2 + (lon_values - obs_coord[1])**2)

# Find the index of the minimum distance
closest_idx = distances.argmin()

# Extract data for the closest point using the index
closest_point_data = era5.isel(index=closest_idx)

# Combine 'time', 'hs', 'tp', and 'wd' into a single DataFrame
combined_df = pd.DataFrame({
    'time': closest_point_data['time'].values,
    'hs': closest_point_data['hs'].values.flatten(),  # Flatten the data
    'tp': closest_point_data['tp'].values.flatten()
    # 'wd': closest_point_data['wd'].values.flatten()
})

# Save the combined DataFrame to a CSV file
dataset_path = r"./Clean/wave.csv"
combined_df.to_csv(dataset_path, index=False)

print("Combined data has been successfully saved to 'wave.csv'.")

Combined data has been successfully saved to 'wave.csv'.
